In [3]:
import pandas as pd
import re


In [5]:
# Charger le fichier Excel
fichier = "../data/2023_01_01_ExportRechLibre.xlsx"  # Remplace par ton chemin exact si besoin
df = pd.read_excel(fichier)

In [7]:
# Ne garder que les colonnes utiles pour ce traitement
df_actes = df[['Identifiant passage unique', 'Actes']].dropna(subset=['Actes'])


In [8]:
# Initialiser un tableau pour les événements extraits
event_log = []

In [9]:
# Regex pour extraire les couples (activité, timestamp)
pattern = r"(.*?)(\(\d{2}/\d{2}/\d{4} \d{2}:\d{2}\))"

In [10]:
# Boucle ligne par ligne
for _, row in df_actes.iterrows():
    case_id = row['Identifiant passage unique']
    texte = row['Actes']

In [12]:
# Boucle ligne par ligne
for _, row in df_actes.iterrows():
    case_id = row['Identifiant passage unique']
    texte = row['Actes']

    # Trouver tous les couples activité + date
    matches = re.findall(pattern, texte)

    for activity_raw, time_str in matches:
        # Nettoyage de l'activité
        activity = activity_raw.strip().lower()
        activity = re.sub(r"[^a-zA-Z0-9éèàçùêîôâ\- ]", "", activity)

        # Timestamp de début
        start_time = pd.to_datetime(time_str.strip("()"), format="%d/%m/%Y %H:%M")

        # Ajouter à l'event log
        event_log.append({
            "case_id": case_id,
            "activity": activity,
            "start_timestamp": start_time
        })

In [13]:
# Conversion en DataFrame
event_log_df = pd.DataFrame(event_log)

# Affichage d’un aperçu
event_log_df.sort_values(by=['case_id', 'start_timestamp']).head(10)

,case_id,activity,start_timestamp
22,569611027,prise de la température,2023-01-02 00:23:00
23,569611027,surveillance scope spo2 2 heures,2023-01-02 00:23:00
24,569611027,prélèvement sécrétions,2023-01-02 00:23:00
25,569611027,résultats des tdr,2023-01-02 00:46:00
26,569611027,dispensation de médicament po,2023-01-02 00:46:00
27,569611030,désinfection plaie,2023-01-02 00:00:00
28,569611030,pose xylocaine,2023-01-02 00:00:00
29,569611030,aide à la suture,2023-01-02 00:43:00
30,569611030,inhalation de meopa pour soins médicaux,2023-01-02 00:43:00
31,569611030,prise en charge en contexte traumatique yyyy011,2023-01-02 09:05:00


In [33]:
from rapidfuzz import process
#Construire un dictionnaire des catégories connues
#Définir le dictionnaire expert
expert_categories = {
    "plaster": "bandage",
    "injection": "injection",
    "inhalation": "inhalation",
    "suture": "suture",
    "surgery": "surgery",
    "bandage": "bandage",
    "cleaning": "cleaning",
    "intubation": "intubation",
    "burns": "burns",
    "catherization": "catherization",
    "drug dispensation": "drug dispensation",
    "surveillance": "monitoring",
    "support": "support"
}
known_activities = list(expert_categories.keys())

def classify_activity(activity):
    try:
        if not isinstance(activity, str) or activity.strip() == "":
            return "inconnu"
        match, score, _ = process.extractOne(activity, known_activities)
        if match and score >= 85:
            return expert_categories[match]
        else:
            return "autre"
    except Exception as e:
        return f"erreur: {e}"

# Appliquer la classification
event_log_df['category'] = event_log_df['activity'].apply(classify_activity)

# Afficher un aperçu
print(event_log_df.head())

     case_id                   activity     start_timestamp category
0  569611249    prise de la température 2023-01-01 11:03:00    autre
1  569611249     mesure ponctuelle spo2 2023-01-01 11:03:00    autre
2  569611249  crp capillaire resultat45 2023-01-01 12:13:00    autre
3  569611249         pose de patch emla 2023-01-01 12:28:00    autre
4  569611259         désinfection plaie 2023-01-01 10:27:00    autre


In [30]:
from rapidfuzz import process